In [2]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler

In [3]:
# path = r"F:\Jyoti Shukla -MS\AVHRR global data (1981-2022)"
# shapefile = r"F:\Jyoti Shukla -MS\State.shp"
path = r"F:\Jyoti Shukla -MS\AVHRR Karnataka data (1981-2022)"
dir_list = os.listdir(path)
len(dir_list)

2066

In [4]:
list_relevant = ['VHP.G04.C07.NF.P1985009.VH.VHI.tif','VHP.G04.C07.NH.P1989001.VH.VHI.tif', 'VHP.G04.C07.NJ.P1995004.VH.VHI.tif','VHP.G04.C07.NL.P2004005.VH.VHI.tif', 'VHP.G04.C07.NL.P2004025.VH.VHI.tif', 'VHP.G04.C07.NL.P2004030.VH.VHI.tif']

In [5]:
def relevant_images_for_mean(filename,path, missing_file):
    dir_list = os.listdir(path)
    relevant_data =[]
    n=0
    for file in dir_list:
        if file == filename:
            break 
        else:            
            if file.endswith(".tif") and missing_file in file:        
                image = os.path.join(path,file)
                ds = gdal.Open(image)
                array = ds.GetRasterBand(1).ReadAsArray()
                relevant_data.append(array)
                n+=1
        # if file == filename:
        #     break
    return relevant_data, n
       

In [191]:
filename = 'VHP.G04.C07.NC.P1982043.VH.VHI.tif'
missing_file = '042'
relevant_data_1, n_1 = relevant_images_for_mean(filename, path, missing_file)
n_1,len(relevant_data_1), relevant_data_1[0].shape

(1, 1, (190, 124))

In [6]:
def array_to_image(array,dtype,outfile,nodata):
    path = r"F:\Jyoti Shukla -MS\AVHRR Karnataka data (1981-2022)\VHP.G04.C07.NC.P1982047.VH.VHI.tif"  
    image=gdal.Open(path)     
    trans = image.GetGeoTransform()
    proj = image.GetProjection()
    #nodata= image.GetRasterBand(1).GetNoDataValue()
    #out ="testimage.tif"
    outdriver = gdal.GetDriverByName("GTIFF")
    outdata = outdriver.Create(str(outfile),image.RasterXSize, image.RasterYSize, 1,gdal.GDT_Float32)
    outdata.GetRasterBand(1).WriteArray(array)
    outdata.GetRasterBand(1).SetNoDataValue(nodata)
    outdata.SetGeoTransform(trans)
    outdata.SetProjection(proj)
    outdata=None        

In [7]:
def mean_filling(relevant_data, outfile):
    arr = np.zeros((190,124)).astype("float32")
    n = np.zeros((arr.shape))
    #val = np.zeros((a.shape))
    #a = im

    for img in relevant_data_1:    
        for i in range(img.shape[0]):
            for j in range(img.shape[1]):
                if img[i][j] >-0.1:
                    arr[i][j] += img[i][j]
                    n[i][j]+=1
                elif img[i][j] == -9999:
                    arr[i][j] = img[i][j]
                    n[i][j] =1
                else:
                    arr[i][j] = -3.4e+38
                    n[i][j] = 1                
    arr/= n
    nodata = -3.4e+38
    array_to_image(arr, "float32",outfile, nodata)
    return arr

In [79]:
outfile = r"F:\Jyoti Shukla -MS\AVHRR Karnataka data (1981-2022)"+"/VHP.G04.C07.NC.P1984050.VH.VHI.tif"
arr = mean_filling(relevant_data_1, outfile)
arr

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [8]:
#1982-042
missing_file = '042'
relevant_data_1, n_1 = relevant_images_for_mean('VHP.G04.C07.NC.P1982043.VH.VHI.tif',path, missing_file)
print(n_1)
array_1 = mean_filling(relevant_data_1, r"F:\Jyoti Shukla -MS\AVHRR Karnataka data (1981-2022)"+"/VHP.G04.C07.NC.P1982042.VH.VHI.tif")

1


In [12]:
#2000-014
missing_file = '014'
relevant_data_2, n_2 = relevant_images_for_mean('VHP.G04.C07.NJ.P2000015.VH.VHI.tif', path, missing_file)
print(n_2)
array_1 = mean_filling(relevant_data_2, r"F:\Jyoti Shukla -MS\AVHRR Karnataka data (1981-2022)"+"/VHP.G04.C07.NJ.P2000014.VH.VHI.tif")

18


In [ ]:
#1984-052
relevant_data_2, n_2 = relevant_images_for_mean('VHP.G04.C07.NF.P1985009.VH.VHI.tif', dir_list,path)
print(n_2)
array_1 = mean_filling(relevant_data_2, r"F:\Jyoti Shukla -MS\AVHRR Karnataka data (1981-2022)"+"/VHP.G04.C07.NC.P1984051.VH.VHI.tif")

#1984-052
relevant_data_2, n_2 = relevant_images_for_mean('VHP.G04.C07.NF.P1985009.VH.VHI.tif', dir_list,path)
print(n_2)
array_1 = mean_filling(relevant_data_2, r"F:\Jyoti Shukla -MS\AVHRR Karnataka data (1981-2022)"+"/VHP.G04.C07.NC.P1984051.VH.VHI.tif")


In [10]:
outname = ['VHP.G04.C07.NC.P1984050.VH.VHI.tif','VHP.G04.C07.NC.P1984051.VH.VHI.tif','VHP.G04.C07.NC.P1984052.VH.VHI.tif', 'VHP.G04.C07.NF.P1985001.VH.VHI.tif', 'VHP.G04.C07.NF.P1985002.VH.VHI.tif','VHP.G04.C07.NF.P1985003.VH.VHI.tif', 'VHP.G04.C07.NF.P1985004.VH.VHI.tif', 'VHP.G04.C07.NF.P1985005.VH.VHI.tif', 'VHP.G04.C07.NF.P1985006.VH.VHI.tif','VHP.G04.C07.NF.P1985007.VH.VHI.tif', 'VHP.G04.C07.NF.P1985008.VH.VHI.tif', 'VHP.G04.C07.NF.P1988046.VH.VHI.tif','VHP.G04.C07.NF.P1988047.VH.VHI.tif','VHP.G04.C07.NF.P1988048.VH.VHI.tif','VHP.G04.C07.NF.P1988049.VH.VHI.tif','VHP.G04.C07.NF.P1988050.VH.VHI.tif','VHP.G04.C07.NF.P1988051.VH.VHI.tif','VHP.G04.C07.NF.P1988052.VH.VHI.tif','VHP.G04.C07.NH.P1994037.VH.VHI.tif','VHP.G04.C07.NH.P1994038.VH.VHI.tif','VHP.G04.C07.NH.P1994039.VH.VHI.tif','VHP.G04.C07.NH.P1994040.VH.VHI.tif','VHP.G04.C07.NH.P1994041.VH.VHI.tif','VHP.G04.C07.NH.P1994042.VH.VHI.tif','VHP.G04.C07.NH.P1994043.VH.VHI.tif','VHP.G04.C07.NH.P1994044.VH.VHI.tif','VHP.G04.C07.NH.P1994045.VH.VHI.tif','VHP.G04.C07.NH.P1994046.VH.VHI.tif','VHP.G04.C07.NH.P1994047.VH.VHI.tif','VHP.G04.C07.NH.P1994048.VH.VHI.tif', 'VHP.G04.C07.NH.P1994049.VH.VHI.tif','VHP.G04.C07.NH.P1994050.VH.VHI.tif','VHP.G04.C07.NH.P1994051.VH.VHI.tif','VHP.G04.C07.NH.P1994052.VH.VHI.tif','VHP.G04.C07.NJ.P1995001.VH.VHI.tif','VHP.G04.C07.NJ.P1995002.VH.VHI.tif','VHP.G04.C07.NJ.P1995003.VH.VHI.tif','VHP.G04.C07.NL.P2004002.VH.VHI.tif','VHP.G04.C07.NL.P2004003.VH.VHI.tif','VHP.G04.C07.NL.P2004004.VH.VHI.tif','VHP.G04.C07.NL.P2004011.VH.VHI.tif','VHP.G04.C07.NL.P2004012.VH.VHI.tif','VHP.G04.C07.NL.P2004013.VH.VHI.tif','VHP.G04.C07.NL.P2004014.VH.VHI.tif','VHP.G04.C07.NL.P2004015.VH.VHI.tif','VHP.G04.C07.NL.P2004016.VH.VHI.tif','VHP.G04.C07.NL.P2004017.VH.VHI.tif','VHP.G04.C07.NL.P2004018.VH.VHI.tif','VHP.G04.C07.NL.P2004019.VH.VHI.tif','VHP.G04.C07.NL.P2004020.VH.VHI.tif','VHP.G04.C07.NL.P2004021.VH.VHI.tif','VHP.G04.C07.NL.P2004022.VH.VHI.tif','VHP.G04.C07.NL.P2004023.VH.VHI.tif','VHP.G04.C07.NL.P2004024.VH.VHI.tif','VHP.G04.C07.NL.P2004029.VH.VHI.tif']
len(outname)

55

In [9]:
arr = ['050','051','052', '001','002','003','004','005','006','007','008','046','047','048','049','050','051','052','037','038','039','040','041','042','043','044','045','046','047','048','049','050','051','052','001','002','003','002','003','004','011','012','013','014','015','016','017','018','019','020','021','022','023','024','029']
len(arr)

55

In [ ]:
for i in range(55):
    if i<12:
        print(outname[i])
    if i>=12 and i<18:
        print(outname[i])
    if i>=18 and i<37:
        print(outname[i])
    if i>=37 and i<40:
        print(outname[i])
    if i>=40 and i<54:
        print(outname[i])
    else:
        print(outname[i])
outname[11],arr[11]

In [11]:
relevant_data = []
n_data = []
arrays = []
for i in range(55):
    if i<12:
        r_data, n_ = relevant_images_for_mean(list_relevant[0],path,arr[i])
        print(n_)
        arr_data = mean_filling(r_data, path+'/'+outname[i])
        relevant_data.append(r_data)
        n_data.append(n_)
    elif i>=12 and i<18:
        r_data, n_ = relevant_images_for_mean(list_relevant[1],path,arr[i])
        print(n_)
        arr_data = mean_filling(r_data, path+'/'+outname[i])
        relevant_data.append(r_data)
        n_data.append(n_)
    
    elif i>=18 and i<37:
        r_data, n_ = relevant_images_for_mean(list_relevant[2],path,arr[i])
        print(n_)
        arr_data = mean_filling(r_data, path+'/'+outname[i])
        relevant_data.append(r_data)
        n_data.append(n_)
    
    elif i>=37 and i<40:
        r_data, n_ = relevant_images_for_mean(list_relevant[3],path,arr[i])
        print(n_)
        arr_data = mean_filling(r_data, path+'/'+outname[i])
        relevant_data.append(r_data)
        n_data.append(n_)

    elif i>=40 and i<54:
        r_data, n_ = relevant_images_for_mean(list_relevant[4],path,arr[i])
        print(n_)
        arr_data = mean_filling(r_data, path+'/'+outname[i])
        relevant_data.append(r_data)
        n_data.append(n_)

    else:
        r_data, n_ = relevant_images_for_mean(list_relevant[5],path,arr[i])
        print(n_)
        arr_data = mean_filling(r_data, path+'/'+outname[i])
        relevant_data.append(r_data)
        n_data.append(n_)

3
3
3
3
3
3
3
3
3
3
3
4
7
7
7
7
7
7
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
13
23
23
23
93
93
45
22
22
22
21
22
22
22
22
22
73
22
22
22
22
22
